In [ ]:
!pip install mediapipe opencv-python

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import os
from google.colab import drive
from scipy.special import comb


In [ ]:
drive.mount('/content/drive',force_remount=True)

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

In [ ]:
def bernstein_poly(i, n, t):
    """The Bernstein polynomial of n, i as a function of t."""
    return comb(n, i) * (t ** (n - i)) * (1 - t) ** i

def bezier_curve(points, nTimes=1000):
    """
    Generate Bezier curves from control points.
    nTimes is the number of time steps.
    """
    nPoints = len(points)
    xPoints = np.array([p[0] for p in points])
    yPoints = np.array([p[1] for p in points])

    t = np.linspace(0.0, 1.0, nTimes)
    polynomial_array = np.array([bernstein_poly(i, nPoints - 1, t) for i in range(0, nPoints)])

    xvals = np.dot(xPoints, polynomial_array)
    yvals = np.dot(yPoints, polynomial_array)

    return list(zip(xvals.astype(int), yvals.astype(int)))

def expand_polygon(points, expansion=10):
    # Simplified version just for demonstration
    if len(points) < 2:
        return points
    expanded_polygon = []
    for i in range(len(points) - 1):
        p1 = np.array(points[i])
        p2 = np.array(points[i + 1])
        vector = p2 - p1
        perpendicular = np.array([-vector[1], vector[0]])
        perpendicular = perpendicular / np.linalg.norm(perpendicular) * expansion
        expanded_polygon.append((p1 - perpendicular).astype(int))
        expanded_polygon.append((p1 + perpendicular).astype(int))
    p2 = points[-1]
    expanded_polygon.append((p2 - perpendicular).astype(int))
    expanded_polygon.append((p2 + perpendicular).astype(int))
    return expanded_polygon

In [ ]:
def process_image(image):
    # Convert image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Initialize MediaPipe Hands
    with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
        results = hands.process(image)

        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]

            h, w, _ = image.shape
            x_min = int(min([lm.x for lm in hand_landmarks.landmark]) * w)
            x_max = int(max([lm.x for lm in hand_landmarks.landmark]) * w)
            y_min = int(min([lm.y for lm in hand_landmarks.landmark]) * h)
            y_max = int(max([lm.y for lm in hand_landmarks.landmark]) * h)

            cropped_image = image[y_min:y_max, x_min:x_max]
            cropped_image = cv2.resize(cropped_image, (256, 256))

            # Create a single-channel mask
            mask = np.zeros(cropped_image.shape[:2], dtype=np.uint8)

            # Convert single-channel mask to BGR
            mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

            # Draw landmarks on the BGR mask
            print(mp_hands.HAND_CONNECTIONS)
            test = []
            for conn in mp_hands.HAND_CONNECTIONS:
              if 0 in conn:
                continue
              test.append(conn)
            mp_drawing.draw_landmarks(
                mask_bgr,
                hand_landmarks,
                test,
                mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=2),
                mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=4)
            )

            landmarks = [lm for lm in hand_landmarks.landmark]
            adjusted_landmarks = [(int(lm.x * 256), int(lm.y * 256)) for lm in landmarks]
            palm = [adjusted_landmarks[i] for i in [0, 2, 5, 9, 13, 17]]
            p = expand_polygon(palm)
            cv2.fillPoly(mask_bgr, [np.array(p, dtype=np.int32)], 255)


            # Extract the single-channel mask from BGR mask
            mask = cv2.cvtColor(mask_bgr, cv2.COLOR_BGR2GRAY)

            kernel = np.ones((15, 15), np.uint8)
            mask = cv2.dilate(mask, kernel, iterations=1)

            # Create a complete black and white image where the hand is white
            # hand_image = np.zeros_like(cropped_image)
            hand_image = np.zeros(cropped_image.shape[:2], dtype=np.uint8)  # Note change here: shape and dtype

            hand_image[mask > 0] = 255  # All channels to 255 where mask is white
    # Display the processed hand image
            plt.figure(figsize=(3, 3))
            plt.imshow(hand_image, cmap='gray')  # Ensure it is displayed as grayscale
            plt.title(f"Processed Image")
            plt.axis('off')
            plt.show()
            return hand_image

        else:
            print("no hand")
            return None

In [ ]:
directory = '/content/drive/MyDrive/test_2'
for filename in os.listdir(directory):
  file_path = os.path.join(directory, filename)
  image = cv2.imread(file_path)
  if image is None:
      continue
  plt.figure(figsize=(3, 3))
  plt.imshow(image, cmap='gray')  # Ensure it is displayed as grayscale
  plt.title(f"Preprocessed  Image")
  plt.axis('off')
  plt.show()

  hand_image = process_image(image)